# Chapter 16 样本筛选

在学习班原课本的美国案例中，前文的股票样本一直是采用美国本国股票全市场样本，并没有依据股票特征对样本进行筛选，而在第十六章开始，探讨极端样本会引发的问题，并提出了两种筛选方案。

而与美国情况不同，中国由于市场的特色，实证文章一直都采取一定的样本筛选的条件。对于股票上的筛选主要分为公司特征和上市时间两个方向。在公司特征中，一般实证文章要求筛除金融和ST股票或者市值排序最后30%的股票，在Size and Value in China一文中解释了原因：在中国市场上，小市值公司大多为具有管理意义的壳公司，而非正常经营的公司，所以剔除这些小市值壳公司有利于分析市场真实的规律。而在上市时间上，一般实证文章只会取上市超过一年（有些是半年，比如要复制的Size and Value in China一文）的股票，这是由于中国市场普遍存在的IPO溢价，会影响市场真实规律。

学习班前面章节的样本，都按照Size and Value in China一文中的样本处理，剔除了每期市值排名末尾30%的股票样本和每期中上市不满半年的股票样本。下面是处理样本选取的程序。

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

#########################################上市时间表格处理
IPO_date = pd.read_csv('上市时间.csv')
IPO_date['date'] = pd.to_datetime(IPO_date['date'])
IPO_date = IPO_date[(IPO_date['type']!=2)&(IPO_date['type']!=8)]
IPO_date['month_num'] = (IPO_date['date'].dt.year-2000)*12+IPO_date['date'].dt.month
IPO_list = IPO_date[['code','month_num']]

def make_codelist1(df):
    '''
    df为包含股票代码和上市月份的面板数据
    输出index为股票代码，columns为月份数的df,0代表在这一期被删去，1代表在这一期被保留
    '''
    X = pd.DataFrame()
    for code in df['code']:
        temp = df[df['code']==code]
        temp = temp.iloc[0,:]
        month = temp['month_num']+6
        x = pd.Series(index = range(1,241),name = code)
        if month <=1:
            x[:] = 1
        elif month>240:
            x[:] = 0
        else:
            x[:] = 0
            x[month-1:] = 1
        X = pd.concat([X,x],axis=1)
    X = X.T
    return X

code_IPO = make_codelist1(IPO_list)   
########################################市值处理
monthly = pd.read_csv('monthly.csv')
Acode = pd.read_csv('Acode.csv')
monthly = pd.merge(monthly,Acode,on='code')
monthly = monthly[(monthly['exchcd']!=2)&(monthly['exchcd']!=8)]
monthly['date'] = pd.to_datetime(monthly['date'])
monthly['month_num'] = (monthly['date'].dt.year-2000)*12+monthly['date'].dt.month
monthly = monthly[(monthly['month_num']>0)&(monthly['month_num']<241)]
monthly = monthly[['code','mktcap','month_num']]
mktcap = pd.pivot(monthly,index='code',columns='month_num',values='mktcap')

def make_codelist2(df):
    '''
    df为透视数据index为股票代码，columns为月份数，value为mktcap
    输出index为股票代码，columns为月份数的df,0代表在这一期被删去，1代表在这一期被保留
    ''' 
    X = df.copy()
    for i in df.columns:
        X[i] = pd.qcut(df[i],[0,0.3,1],labels=False)
    X[X.isna()] = 0
    return X

code_mktcapx =  make_codelist2(mktcap)

##########################################综合两个表
code_IPO = code_IPO.loc[code_mktcap.index]

codelist = code_IPO*code_mktcap
codelist = codelist.dropna()